In [85]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

def splitdata(YEAR):
     
    StaphAureuaDataPath = '/project/biocomplexity/anil/AMR/StaphAureusAntibiogramTimeSpace/RawData/StaphAureua{}.csv'.format(YEAR)
    hsaRegionDataPath = '/home/uzn2up/crosswalks/ZipHsaHrr{}.xls'.format(YEAR)

    StaphAureuaData = pd.read_csv(StaphAureuaDataPath)
    hsaRegionData = pd.read_excel(hsaRegionDataPath)
    
    StaphAureuaData['Zipcode'] = StaphAureuaData['Zipcode'].fillna(0).astype(int)
    StaphAureuaData['Zipcode'] = StaphAureuaData['Zipcode'].astype(str)
    StaphAureuaData['Zipcode'] = StaphAureuaData['Zipcode'].str.zfill(5)
    hsaRegionData['cityAndState'] = hsaRegionData['hsacity'] +' '+ hsaRegionData['hsastate']
    hsaRegionData['zipcode{}'.format(str(YEAR)[-2:])] = hsaRegionData['zipcode{}'.format(str(YEAR)[-2:])].astype(str)
    hsaRegionData['zipcode{}'.format(str(YEAR)[-2:])] = hsaRegionData['zipcode{}'.format(str(YEAR)[-2:])].str.zfill(5)

    hsaDirectoryNew = {}
    for ind in hsaRegionData.index:
        hsaDirectoryNew[hsaRegionData['zipcode{}'.format(str(YEAR)[-2:])][ind]] = hsaRegionData['cityAndState'][ind]
    
    hsaDirectoryNew['94101'] = 'San Francisco CA'
    hsaDirectoryNew['33115'] = 'Miami FL'
    hsaDirectoryNew['40354'] = 'Georgetown KY'
    
    StaphAureuaData['HSA_REGION'] = ''
    for ind in StaphAureuaData.index:
        curZip = StaphAureuaData['Zipcode'][ind]
        region = hsaDirectoryNew.get(curZip)
        StaphAureuaData['HSA_REGION'][ind] = region
    StaphAureuaData.to_csv('region_csv/{}.csv'.format(YEAR),index=False)

In [ ]:
for i in range(1999,2013):
    splitdata(i)

In [110]:
def generateOutputFiles(YEAR):
    
    yearData = pd.read_csv('region_csv/{}.csv'.format(YEAR))
    regionList = yearData['HSA_REGION'].unique()
    regionList = [x for x in regionList if str(x) != 'nan']

    for region in regionList:
        rslt_df = yearData[yearData['HSA_REGION'] == region]
        
        if "/" in region:
            region = region.replace("/", "")
        
        rslt_df.to_csv('splitData/{}{}.csv'.format(region.replace(" ", ""), YEAR),index=False)

In [92]:
for i in range(1999,2013):
    generateOutputFiles(i)

In [88]:
import glob
import os
import pandas as pd
all_files = glob.glob("region_csv/*.csv")
allData = pd.concat((pd.read_csv(f) for f in all_files))

In [89]:
allData.to_csv('alldataregion.csv')

In [90]:
allData

,Organism,PatientAge,PatientGender,PatientLocation,Region,State,Zipcode,SpecimenSource,SpecimenSourceCategory,ReportDate,...,Moxifloxacin,Nitrofurantoin,Oxacillin,Penicillin,Quinupristin/dalfopristin,Rifampin (Rifampicin),Tetracycline,Trimeth/sulfa,Vancomycin,HSA_REGION
0,Staphylococcus aureus,89.0,M,Inpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,2012-01-03,...,R,S,R,NaN,NaN,S,S,S,S,NaN
1,Staphylococcus aureus,77.0,M,Inpatient,MOUNTAIN,Arizona,85001,Bronchial Alveolar Lavage,Respiratory-Lower,2012-01-03,...,S,S,S,NaN,NaN,S,S,S,S,Phoenix AZ
2,Staphylococcus aureus,89.0,F,Outpatient,MOUNTAIN,Arizona,85301,Wound - Unspecified,Wound,2012-01-04,...,S,S,S,NaN,NaN,S,S,S,S,Phoenix AZ
3,Staphylococcus aureus,25.0,M,Inpatient,MOUNTAIN,Arizona,85301,Bronchial Alveolar Lavage,Respiratory-Lower,2012-01-04,...,S,S,S,NaN,NaN,S,S,S,S,Phoenix AZ
4,Staphylococcus aureus,60.0,F,Outpatient,MOUNTAIN,New Mexico,86040,Wound - Unspecified,Wound,2012-01-06,...,R,S,R,NaN,NaN,S,S,S,S,Page AZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153808,Staphylococcus aureus,65.0,F,Outpatient,PACIFIC,California,90401,Urine,Urine,2011-07-08,...,NaN,S,R,R,NaN,S,S,S,S,Santa Monica CA
153809,Staphylococcus aureus,69.0,F,Outpatient,PACIFIC,California,90401,Sinus,Respiratory-Upper,2011-07-08,...,NaN,NaN,S,R,NaN,S,S,S,S,Santa Monica CA
153810,Staphylococcus aureus,16.0,M,Outpatient,PACIFIC,California,90401,"Ear, External",Respiratory-Upper,2011-07-12,...,NaN,NaN,S,S,NaN,S,S,S,S,Santa Monica CA
153811,Staphylococcus aureus,78.0,M,Inpatient,PACIFIC,California,90401,Blood,Blood,2011-06-20,...,NaN,NaN,S,R,NaN,S,S,S,S,Santa Monica CA


In [34]:
regionList = allData['HSA_REGION'].unique()
regionList = [x for x in regionList if str(x) != 'nan']


In [35]:
regionList

['Phoenix AZ',
 'Page AZ',
 'Mesa AZ',
 'Burlington VT',
 'Seattle WA',
 'Wichita KS',
 'Portland OR',
 'Tulsa OK',
 'Grand Rapids MI',
 'Chula Vista CA',
 'La Jolla CA',
 'St. Louis MO',
 'Allentown PA',
 'Boston MA',
 'Newberg OR',
 'Seaside OR',
 'Pocahontas AR',
 'Houston TX',
 'Charlotte NC',
 'Monroe NC',
 'Shelby NC',
 'Syracuse NY',
 'Santa Monica CA',
 'Mayfield Heights OH',
 'Cleveland OH',
 'Euclid OH',
 'Morgantown WV',
 'Albuquerque NM',
 'Maryville IL',
 'White Plains NY',
 'Sacramento CA',
 'Torrance CA',
 'Fountain Valley CA',
 'Bangor ME',
 'Ridgewood NJ',
 'Grand Forks ND',
 'Valdese NC',
 'Morganton NC',
 'Kennewick WA',
 'Danville PA',
 'Wilkes-Barre PA',
 'Dallas TX',
 'Chicago IL',
 'Coral Gables FL',
 'South Miami FL',
 'Miami FL',
 'Tavernier FL',
 'Birmingham AL',
 'Takoma Park MD',
 'Washington DC',
 'Rockville MD',
 'Madison WI',
 'Burlington MA',
 'Pasco WA',
 'Charlottesville VA',
 'Raleigh NC',
 'Greensboro NC',
 'Jacksonville FL',
 'Fernandina Beach FL',


In [36]:
len(regionList)

203

In [51]:
allData[(allData['Zipcode'] == 0)]

,Unnamed: 0,Organism,PatientAge,PatientGender,PatientLocation,Region,State,Zipcode,SpecimenSource,SpecimenSourceCategory,...,Moxifloxacin,Nitrofurantoin,Oxacillin,Penicillin,Quinupristin/dalfopristin,Rifampin (Rifampicin),Tetracycline,Trimeth/sulfa,Vancomycin,HSA_REGION
0,0,Staphylococcus aureus,89.0,M,Inpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,...,R,S,R,NaN,NaN,S,S,S,S,NaN
5,5,Staphylococcus aureus,89.0,F,Outpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,...,S,S,S,NaN,NaN,S,S,S,S,NaN
6,6,Staphylococcus aureus,1.0,F,Outpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,...,NaN,NaN,S,NaN,NaN,NaN,S,S,S,NaN
14,14,Staphylococcus aureus,72.0,F,Outpatient,MOUNTAIN,Arizona,0,Bronchial Washing,Respiratory-Lower,...,S,S,S,NaN,NaN,S,S,S,S,NaN
61,61,Staphylococcus aureus,78.0,M,Outpatient,WEST SOUTH CENTRAL,Texas,0,Wound - Unspecified,Wound,...,NaN,NaN,S,R,NaN,S,S,S,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153729,153729,Staphylococcus aureus,31.0,M,Inpatient,SOUTH ATLANTIC,Maryland,0,Wound - Unspecified,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153730,153730,Staphylococcus aureus,38.0,F,Inpatient,SOUTH ATLANTIC,Florida,0,Abscess,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153740,153740,Staphylococcus aureus,42.0,M,Inpatient,SOUTH ATLANTIC,Florida,0,Abscess,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153743,153743,Staphylococcus aureus,56.0,F,Inpatient,SOUTH ATLANTIC,Maryland,0,Wound - Unspecified,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN


In [63]:
allData[allData['HSA_REGION'].isna()]

,Unnamed: 0,Organism,PatientAge,PatientGender,PatientLocation,Region,State,Zipcode,SpecimenSource,SpecimenSourceCategory,...,Moxifloxacin,Nitrofurantoin,Oxacillin,Penicillin,Quinupristin/dalfopristin,Rifampin (Rifampicin),Tetracycline,Trimeth/sulfa,Vancomycin,HSA_REGION
0,0,Staphylococcus aureus,89.0,M,Inpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,...,R,S,R,NaN,NaN,S,S,S,S,NaN
5,5,Staphylococcus aureus,89.0,F,Outpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,...,S,S,S,NaN,NaN,S,S,S,S,NaN
6,6,Staphylococcus aureus,1.0,F,Outpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,...,NaN,NaN,S,NaN,NaN,NaN,S,S,S,NaN
14,14,Staphylococcus aureus,72.0,F,Outpatient,MOUNTAIN,Arizona,0,Bronchial Washing,Respiratory-Lower,...,S,S,S,NaN,NaN,S,S,S,S,NaN
61,61,Staphylococcus aureus,78.0,M,Outpatient,WEST SOUTH CENTRAL,Texas,0,Wound - Unspecified,Wound,...,NaN,NaN,S,R,NaN,S,S,S,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153729,153729,Staphylococcus aureus,31.0,M,Inpatient,SOUTH ATLANTIC,Maryland,0,Wound - Unspecified,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153730,153730,Staphylococcus aureus,38.0,F,Inpatient,SOUTH ATLANTIC,Florida,0,Abscess,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153740,153740,Staphylococcus aureus,42.0,M,Inpatient,SOUTH ATLANTIC,Florida,0,Abscess,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153743,153743,Staphylococcus aureus,56.0,F,Inpatient,SOUTH ATLANTIC,Maryland,0,Wound - Unspecified,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN


In [64]:
allData[allData['Zipcode'] == 0]

,Unnamed: 0,Organism,PatientAge,PatientGender,PatientLocation,Region,State,Zipcode,SpecimenSource,SpecimenSourceCategory,...,Moxifloxacin,Nitrofurantoin,Oxacillin,Penicillin,Quinupristin/dalfopristin,Rifampin (Rifampicin),Tetracycline,Trimeth/sulfa,Vancomycin,HSA_REGION
0,0,Staphylococcus aureus,89.0,M,Inpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,...,R,S,R,NaN,NaN,S,S,S,S,NaN
5,5,Staphylococcus aureus,89.0,F,Outpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,...,S,S,S,NaN,NaN,S,S,S,S,NaN
6,6,Staphylococcus aureus,1.0,F,Outpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,...,NaN,NaN,S,NaN,NaN,NaN,S,S,S,NaN
14,14,Staphylococcus aureus,72.0,F,Outpatient,MOUNTAIN,Arizona,0,Bronchial Washing,Respiratory-Lower,...,S,S,S,NaN,NaN,S,S,S,S,NaN
61,61,Staphylococcus aureus,78.0,M,Outpatient,WEST SOUTH CENTRAL,Texas,0,Wound - Unspecified,Wound,...,NaN,NaN,S,R,NaN,S,S,S,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153729,153729,Staphylococcus aureus,31.0,M,Inpatient,SOUTH ATLANTIC,Maryland,0,Wound - Unspecified,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153730,153730,Staphylococcus aureus,38.0,F,Inpatient,SOUTH ATLANTIC,Florida,0,Abscess,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153740,153740,Staphylococcus aureus,42.0,M,Inpatient,SOUTH ATLANTIC,Florida,0,Abscess,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153743,153743,Staphylococcus aureus,56.0,F,Inpatient,SOUTH ATLANTIC,Maryland,0,Wound - Unspecified,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN


In [42]:
allData[(allData['HSA_REGION'].isna()) & (allData['Zipcode'] == 0)]

,Unnamed: 0,Organism,PatientAge,PatientGender,PatientLocation,Region,State,Zipcode,SpecimenSource,SpecimenSourceCategory,...,Moxifloxacin,Nitrofurantoin,Oxacillin,Penicillin,Quinupristin/dalfopristin,Rifampin (Rifampicin),Tetracycline,Trimeth/sulfa,Vancomycin,HSA_REGION
0,0,Staphylococcus aureus,89.0,M,Inpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,...,R,S,R,NaN,NaN,S,S,S,S,NaN
5,5,Staphylococcus aureus,89.0,F,Outpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,...,S,S,S,NaN,NaN,S,S,S,S,NaN
6,6,Staphylococcus aureus,1.0,F,Outpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,...,NaN,NaN,S,NaN,NaN,NaN,S,S,S,NaN
14,14,Staphylococcus aureus,72.0,F,Outpatient,MOUNTAIN,Arizona,0,Bronchial Washing,Respiratory-Lower,...,S,S,S,NaN,NaN,S,S,S,S,NaN
61,61,Staphylococcus aureus,78.0,M,Outpatient,WEST SOUTH CENTRAL,Texas,0,Wound - Unspecified,Wound,...,NaN,NaN,S,R,NaN,S,S,S,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153729,153729,Staphylococcus aureus,31.0,M,Inpatient,SOUTH ATLANTIC,Maryland,0,Wound - Unspecified,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153730,153730,Staphylococcus aureus,38.0,F,Inpatient,SOUTH ATLANTIC,Florida,0,Abscess,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153740,153740,Staphylococcus aureus,42.0,M,Inpatient,SOUTH ATLANTIC,Florida,0,Abscess,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153743,153743,Staphylococcus aureus,56.0,F,Inpatient,SOUTH ATLANTIC,Maryland,0,Wound - Unspecified,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN


In [45]:
missZip = allData[(allData['HSA_REGION'].isna()) & (allData['Zipcode'] != 0)]

missZip['Zipcode'].unique()

In [ ]:
#94101 - San Francisco CA
#40354 - Georgetown KY
#33115 - Miami FL

In [50]:
missZip[missZip['Zipcode'] == 40354]

,Unnamed: 0,Organism,PatientAge,PatientGender,PatientLocation,Region,State,Zipcode,SpecimenSource,SpecimenSourceCategory,...,Moxifloxacin,Nitrofurantoin,Oxacillin,Penicillin,Quinupristin/dalfopristin,Rifampin (Rifampicin),Tetracycline,Trimeth/sulfa,Vancomycin,HSA_REGION
6842,6842,Staphylococcus aureus,24.0,F,Outpatient,EAST SOUTH CENTRAL,Kentucky,40354,Urine,Urine,...,NaN,S,S,R,NaN,S,S,S,S,NaN
11085,11085,Staphylococcus aureus,25.0,F,Outpatient,EAST SOUTH CENTRAL,Kentucky,40354,Wound - Unspecified,Wound,...,R,NaN,R,R,NaN,S,S,S,S,NaN
16373,16373,Staphylococcus aureus,48.0,M,Outpatient,EAST SOUTH CENTRAL,Kentucky,40354,Wound - Unspecified,Wound,...,S,NaN,R,R,NaN,S,S,S,S,NaN
16946,16946,Staphylococcus aureus,41.0,F,Outpatient,EAST SOUTH CENTRAL,Kentucky,40354,Wound - Unspecified,Wound,...,S,NaN,S,R,NaN,S,S,S,S,NaN
18984,18984,Staphylococcus aureus,19.0,M,Outpatient,EAST SOUTH CENTRAL,Kentucky,40354,Abdomen,Wound,...,S,NaN,S,R,NaN,S,S,S,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137271,137271,Staphylococcus aureus,90.0,F,Outpatient,EAST SOUTH CENTRAL,Kentucky,40354,Blood,Blood,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
142184,142184,Staphylococcus aureus,40.0,M,Inpatient,EAST SOUTH CENTRAL,Kentucky,40354,Abscess,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
145718,145718,Staphylococcus aureus,90.0,F,Inpatient,EAST SOUTH CENTRAL,Kentucky,40354,Wound - Unspecified,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
148451,148451,Staphylococcus aureus,22.0,M,Outpatient,EAST SOUTH CENTRAL,Kentucky,40354,Wound - Unspecified,Wound,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN


In [49]:
missZip

,Unnamed: 0,Organism,PatientAge,PatientGender,PatientLocation,Region,State,Zipcode,SpecimenSource,SpecimenSourceCategory,...,Moxifloxacin,Nitrofurantoin,Oxacillin,Penicillin,Quinupristin/dalfopristin,Rifampin (Rifampicin),Tetracycline,Trimeth/sulfa,Vancomycin,HSA_REGION
100,100,Staphylococcus aureus,64.0,M,Outpatient,PACIFIC,California,94101,Wound - Unspecified,Wound,...,NaN,NaN,R,R,S,S,S,S,S,NaN
101,101,Staphylococcus aureus,23.0,M,Outpatient,PACIFIC,California,94101,Sputum,Respiratory-Lower,...,NaN,NaN,R,R,S,S,S,S,S,NaN
102,102,Staphylococcus aureus,51.0,M,Inpatient,PACIFIC,California,94101,Abscess,Wound,...,NaN,NaN,S,NaN,S,S,S,S,S,NaN
103,103,Staphylococcus aureus,47.0,M,Inpatient,PACIFIC,California,94101,Bronchial Alveolar Lavage,Respiratory-Lower,...,NaN,NaN,S,R,S,S,S,S,S,NaN
104,104,Staphylococcus aureus,50.0,M,Outpatient,PACIFIC,California,94101,Sinus,Respiratory-Upper,...,NaN,NaN,S,R,S,S,S,S,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153803,153803,Staphylococcus aureus,45.0,M,Outpatient,PACIFIC,California,94101,Sputum,Respiratory-Lower,...,NaN,NaN,R,R,S,S,R,R,S,NaN
153804,153804,Staphylococcus aureus,27.0,M,Outpatient,PACIFIC,California,94101,Sputum,Respiratory-Lower,...,NaN,NaN,R,R,S,S,S,S,S,NaN
153805,153805,Staphylococcus aureus,40.0,F,Outpatient,PACIFIC,California,94101,Sinus,Respiratory-Upper,...,NaN,NaN,S,R,S,R,R,R,S,NaN
153806,153806,Staphylococcus aureus,6.0,F,Outpatient,PACIFIC,California,94101,Abscess,Wound,...,NaN,NaN,S,R,S,S,S,S,S,NaN


In [93]:
tmp = pd.read_csv('splitData/PittsburghPA2000.csv')

In [94]:
tmp

,Organism,PatientAge,PatientGender,PatientLocation,Region,State,Zipcode,SpecimenSource,SpecimenSourceCategory,ReportDate,...,Moxifloxacin,Nitrofurantoin,Oxacillin,Penicillin,Quinupristin/dalfopristin,Rifampin (Rifampicin),Tetracycline,Trimeth/sulfa,Vancomycin,HSA_REGION
0,Staphylococcus aureus,69,M,Inpatient - ICU,MID ATLANTIC,Pennsylvania,15219,Sputum,Respiratory-Lower,2000-12-17,...,NaN,S,R,R,NaN,S,S,S,S,Pittsburgh PA
1,Staphylococcus aureus,69,F,Inpatient - ICU,MID ATLANTIC,Pennsylvania,15219,Sputum,Respiratory-Lower,2000-12-24,...,NaN,S,R,R,NaN,R,S,S,S,Pittsburgh PA
2,Staphylococcus aureus,62,M,Outpatient,MID ATLANTIC,Pennsylvania,15219,Wound - Unspecified,Wound,2000-12-24,...,NaN,S,S,R,NaN,S,S,S,S,Pittsburgh PA
3,Staphylococcus aureus,67,M,Inpatient,MID ATLANTIC,Pennsylvania,15219,Urine,Urine,2000-12-26,...,NaN,S,R,R,NaN,S,S,S,S,Pittsburgh PA
4,Staphylococcus aureus,43,F,Outpatient,MID ATLANTIC,Pennsylvania,15219,Wound - Unspecified,Wound,2000-02-05,...,NaN,S,S,R,NaN,S,S,S,S,Pittsburgh PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,Staphylococcus aureus,86,M,Inpatient,MID ATLANTIC,Pennsylvania,15219,Catheterized Urine,Urine,2000-04-06,...,NaN,S,R,R,NaN,S,R,R,S,Pittsburgh PA
2024,Staphylococcus aureus,62,M,Outpatient,MID ATLANTIC,Pennsylvania,15238,Wound - Unspecified,Wound,2000-04-09,...,NaN,S,S,S,NaN,S,S,S,S,Pittsburgh PA
2025,Staphylococcus aureus,86,F,Inpatient,MID ATLANTIC,Pennsylvania,15219,Perirectal Abscess,Wound,2000-04-09,...,NaN,S,R,R,NaN,S,S,S,S,Pittsburgh PA
2026,Staphylococcus aureus,47,M,Nursing Home,MID ATLANTIC,Pennsylvania,15219,Sputum,Respiratory-Lower,2000-04-08,...,NaN,S,S,R,NaN,S,S,S,S,Pittsburgh PA


In [95]:
allData

,Organism,PatientAge,PatientGender,PatientLocation,Region,State,Zipcode,SpecimenSource,SpecimenSourceCategory,ReportDate,...,Moxifloxacin,Nitrofurantoin,Oxacillin,Penicillin,Quinupristin/dalfopristin,Rifampin (Rifampicin),Tetracycline,Trimeth/sulfa,Vancomycin,HSA_REGION
0,Staphylococcus aureus,89.0,M,Inpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,2012-01-03,...,R,S,R,NaN,NaN,S,S,S,S,NaN
1,Staphylococcus aureus,77.0,M,Inpatient,MOUNTAIN,Arizona,85001,Bronchial Alveolar Lavage,Respiratory-Lower,2012-01-03,...,S,S,S,NaN,NaN,S,S,S,S,Phoenix AZ
2,Staphylococcus aureus,89.0,F,Outpatient,MOUNTAIN,Arizona,85301,Wound - Unspecified,Wound,2012-01-04,...,S,S,S,NaN,NaN,S,S,S,S,Phoenix AZ
3,Staphylococcus aureus,25.0,M,Inpatient,MOUNTAIN,Arizona,85301,Bronchial Alveolar Lavage,Respiratory-Lower,2012-01-04,...,S,S,S,NaN,NaN,S,S,S,S,Phoenix AZ
4,Staphylococcus aureus,60.0,F,Outpatient,MOUNTAIN,New Mexico,86040,Wound - Unspecified,Wound,2012-01-06,...,R,S,R,NaN,NaN,S,S,S,S,Page AZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153808,Staphylococcus aureus,65.0,F,Outpatient,PACIFIC,California,90401,Urine,Urine,2011-07-08,...,NaN,S,R,R,NaN,S,S,S,S,Santa Monica CA
153809,Staphylococcus aureus,69.0,F,Outpatient,PACIFIC,California,90401,Sinus,Respiratory-Upper,2011-07-08,...,NaN,NaN,S,R,NaN,S,S,S,S,Santa Monica CA
153810,Staphylococcus aureus,16.0,M,Outpatient,PACIFIC,California,90401,"Ear, External",Respiratory-Upper,2011-07-12,...,NaN,NaN,S,S,NaN,S,S,S,S,Santa Monica CA
153811,Staphylococcus aureus,78.0,M,Inpatient,PACIFIC,California,90401,Blood,Blood,2011-06-20,...,NaN,NaN,S,R,NaN,S,S,S,S,Santa Monica CA


In [96]:
allData[allData['HSA_REGION'].isna()]

,Organism,PatientAge,PatientGender,PatientLocation,Region,State,Zipcode,SpecimenSource,SpecimenSourceCategory,ReportDate,...,Moxifloxacin,Nitrofurantoin,Oxacillin,Penicillin,Quinupristin/dalfopristin,Rifampin (Rifampicin),Tetracycline,Trimeth/sulfa,Vancomycin,HSA_REGION
0,Staphylococcus aureus,89.0,M,Inpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,2012-01-03,...,R,S,R,NaN,NaN,S,S,S,S,NaN
5,Staphylococcus aureus,89.0,F,Outpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,2012-01-07,...,S,S,S,NaN,NaN,S,S,S,S,NaN
6,Staphylococcus aureus,1.0,F,Outpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,2012-01-08,...,NaN,NaN,S,NaN,NaN,NaN,S,S,S,NaN
14,Staphylococcus aureus,72.0,F,Outpatient,MOUNTAIN,Arizona,0,Bronchial Washing,Respiratory-Lower,2012-01-16,...,S,S,S,NaN,NaN,S,S,S,S,NaN
61,Staphylococcus aureus,78.0,M,Outpatient,WEST SOUTH CENTRAL,Texas,0,Wound - Unspecified,Wound,2012-02-09,...,NaN,NaN,S,R,NaN,S,S,S,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153729,Staphylococcus aureus,31.0,M,Inpatient,SOUTH ATLANTIC,Maryland,0,Wound - Unspecified,Wound,2011-06-26,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153730,Staphylococcus aureus,38.0,F,Inpatient,SOUTH ATLANTIC,Florida,0,Abscess,Wound,2011-07-01,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153740,Staphylococcus aureus,42.0,M,Inpatient,SOUTH ATLANTIC,Florida,0,Abscess,Wound,2011-07-04,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153743,Staphylococcus aureus,56.0,F,Inpatient,SOUTH ATLANTIC,Maryland,0,Wound - Unspecified,Wound,2011-06-09,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN


In [97]:
allData[allData['Zipcode'] == 0]

,Organism,PatientAge,PatientGender,PatientLocation,Region,State,Zipcode,SpecimenSource,SpecimenSourceCategory,ReportDate,...,Moxifloxacin,Nitrofurantoin,Oxacillin,Penicillin,Quinupristin/dalfopristin,Rifampin (Rifampicin),Tetracycline,Trimeth/sulfa,Vancomycin,HSA_REGION
0,Staphylococcus aureus,89.0,M,Inpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,2012-01-03,...,R,S,R,NaN,NaN,S,S,S,S,NaN
5,Staphylococcus aureus,89.0,F,Outpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,2012-01-07,...,S,S,S,NaN,NaN,S,S,S,S,NaN
6,Staphylococcus aureus,1.0,F,Outpatient,MOUNTAIN,Arizona,0,Wound - Unspecified,Wound,2012-01-08,...,NaN,NaN,S,NaN,NaN,NaN,S,S,S,NaN
14,Staphylococcus aureus,72.0,F,Outpatient,MOUNTAIN,Arizona,0,Bronchial Washing,Respiratory-Lower,2012-01-16,...,S,S,S,NaN,NaN,S,S,S,S,NaN
61,Staphylococcus aureus,78.0,M,Outpatient,WEST SOUTH CENTRAL,Texas,0,Wound - Unspecified,Wound,2012-02-09,...,NaN,NaN,S,R,NaN,S,S,S,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153729,Staphylococcus aureus,31.0,M,Inpatient,SOUTH ATLANTIC,Maryland,0,Wound - Unspecified,Wound,2011-06-26,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153730,Staphylococcus aureus,38.0,F,Inpatient,SOUTH ATLANTIC,Florida,0,Abscess,Wound,2011-07-01,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153740,Staphylococcus aureus,42.0,M,Inpatient,SOUTH ATLANTIC,Florida,0,Abscess,Wound,2011-07-04,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN
153743,Staphylococcus aureus,56.0,F,Inpatient,SOUTH ATLANTIC,Maryland,0,Wound - Unspecified,Wound,2011-06-09,...,NaN,NaN,R,R,NaN,S,S,S,S,NaN


In [104]:
import glob
import os
import pandas as pd
all_files = glob.glob("/project/biocomplexity/anil/AMR/StaphAureusAntibiogramTimeSpace/RawData/*.csv")
allData2 = pd.concat((pd.read_csv(f) for f in all_files))

In [108]:
allData2[allData2['Zipcode'].isna()]['State'].value_counts()

Utah              10846
North Carolina     8144
Oklahoma           5742
Arizona            3022
Texas              2753
Ohio               1054
Florida             838
Oregon              549
Maryland            547
California          428
New Mexico          418
Michigan            402
New York            152
Washington, DC       85
Pennsylvania         65
New Jersey           36
Massachusetts        26
Alabama              11
Tennessee            10
Missouri              3
Virginia              2
Name: State, dtype: int64

In [109]:
allData2['State'].value_counts()

Florida           233711
California        211101
North Carolina    199350
Washington        162136
Pennsylvania      145841
Texas             126610
Oregon            122801
Michigan          101888
New York           99941
Oklahoma           96568
Ohio               76322
Tennessee          71629
Arizona            65258
Illinois           54155
Missouri           53009
New Jersey         46846
Maryland           45288
Louisana           45156
Massachusetts      42648
Delaware           42165
Kansas             41765
Virginia           30931
Nevada             28258
Alabama            26926
Nebraska           24018
Vermont            23036
South Carolina     21618
Wisconsin          21176
Kentucky           19806
West Virginia      17676
Washington, DC     17099
Utah               16363
Minnesota          13952
Rhode Island       12888
Georgia            11715
New Mexico         11101
Indiana            10726
North Dakota        9702
Hawaii              9681
Maine               9344
